In [1]:
# External libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.style as style
import cv2
import time
import os
from utility.utils import *
import re
import h5py
import tables

# Own Libraries
from utility.utils import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

# CATALOG CREATION

In [ ]:
DB = pd.read_csv("DATA/lunar_crater_database_robbins_2018.csv", sep=",")
DB['DIAM_CIRC_IMG'] *= 0.5

def craters_from_HLL(H,lat,lon, filter = 1):
    t = find_dteta(H)  # d/R, where d^2 is the footprint
    lat_bounds, lon_bounds = (np.array([lat - t/2, lat + t/2]), np.array([lon - t/2, lon + t/2]) )
    df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME="ROBBINS")
    df.rename(columns={'Diam':'r'}, inplace=True)
    df = df[ df.r > filter]        # km > filter
    df.r = df.r * 1/deg2km    #km2deg
    # Now it's all expressed in deg!
    return df

def HLL_catalog(df_mat):
    H = find_all_triplets(df_mat)
    Q = pd.DataFrame(H, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    # Retrasform radius to km:
    Q.r1 *= deg2km
    Q.r2 *= deg2km
    Q.r3 *= deg2km
    Q.rename( columns={'x1':'lon1','x2':'lon2','x3':'lon3','y1':'lat1','y2':'lat2','y3':'lat3'}, inplace=True)
    return Q

In [ ]:
lon, lat, H = -180, 3, 50
%cd DATA/CATALOG
for i in range(3):
    df = craters_from_HLL(H,lat,lon)
    df_mat = np.array(df)
    cat = HLL_catalog(df_mat)

    # cat.to_hdf(f'test_lat_{lat}_lon_{lon}.h5', key='df', format='table')
    cat.to_hdf(f'part_{i+1}.h5',key='df', format='table', index=False)
    lon+=2

In [ ]:
# LOAD ALL H5:
import glob 
dict = {}
for elem in glob.glob(f"/home/sirbastiano/Desktop/Python Projects/TRN/DATA/CATALOG/*"):
    txt = elem             # stringa completa
    name  = txt.split('/')[-1]
    t = txt.split('_')[1] # numero
    t = t.split('.')[0]
    dict[t] = name

In [ ]:
dfs=[]
for file in dict:
    h = pd.HDFStore(dict[str(file)])
    dfs.append(h['df'])

df = pd.concat(dfs)



# Merge H+L

In [ ]:
H = pd.read_csv('DATA/HeadCraters.csv')
L = pd.read_csv('DATA/LROCCraters.csv')

In [ ]:
H.rename( columns={'Lon':'lon','Lat':'lat','Diam_km':'diam' }, inplace=True)
H.head(2)

In [ ]:
L.rename(columns={'Long':'lon', 'Lat':'lat','Diameter (km)':'diam' }, inplace=True)
L.head(2)

In [ ]:
lon1 = H.lon
lon2= L.lon

lat1= H.lat
lat2=L.lat

diam1=H.diam
diam2=L.diam 

In [ ]:
lon1 =  np.array(lon1)
lon1 = np.vstack(lon1)
lon2 =  np.array(lon2)
lon2 = np.vstack(lon2)

lat1 =  np.array(lat1)
lat1 = np.vstack(lat1)
lat2 =  np.array(lat2)
lat2 = np.vstack(lat2)

diam1 =  np.array(diam1)
diam1 = np.vstack(diam1)
diam2 =  np.array(diam2)
diam2 = np.vstack(diam2)


LON = np.vstack([lon1,lon2]) 
LAT = np.vstack([lat1,lat2]) 
DIAM = np.vstack([diam1,diam2]) 

DATA = np.hstack([LON,LAT,DIAM])

In [ ]:
H_L = pd.DataFrame( DATA, columns=['lon','lat','diam'])
H_L

In [ ]:
H_L.to_csv('H_L_combined.csv', index=False)

# CATALOG CREATION TRN

In [2]:
%cd DATA/CATALOG

/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/CATALOG


In [3]:
DB = pd.read_csv("/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/H_L_combined.csv", sep=",")

def craters_from_HLL(H,lat,lon, filter = 0):
    t = find_dteta(H)  # d/R, where d^2 is the footprint
    lat_bounds, lon_bounds = (np.array([lat - t/2, lat + t/2]), np.array([lon - t/2, lon + t/2]) )
    df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME="COMBINED")
    if df is not None:
        df.rename(columns={'Diam':'r'}, inplace=True)
        df.r = df.r * 0.5/deg2km    #km2deg, diam to radius
        # Now it's all expressed in deg!
        return df
    else:return None

def HLL_catalog(df_mat):
    H = find_all_triplets(df_mat)
    Q = pd.DataFrame(H, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    # Retrasform radius to km:
    Q.r1 *= deg2km
    Q.r2 *= deg2km
    Q.r3 *= deg2km
    Q.rename( columns={'x1':'lon1','x2':'lon2','x3':'lon3','y1':'lat1','y2':'lat2','y3':'lat3'}, inplace=True)
    return Q

In [4]:
lon, lat, H = -180, -90, 60
while lat < 90:
    lon = -180
    while lon < 180:
        df = craters_from_HLL(H,lat,lon)
        if df is not None:
            df_mat = np.array(df)
            cat = HLL_catalog(df_mat)
            cat.to_csv(f'cat_lat_{lat}_lon_{lon}.csv')

        lon+=3
    lat+=3

██████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |██████████████████████████████████████

# MERGE

In [5]:
# LOAD ALL H5:
import glob
names =  glob.glob(f"/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/CATALOG/*")

In [7]:
dfs=[]
for name in names:
    df = pd.read_csv(name)
    dfs.append(df)
# Merging, removing duplicates and resetting index:
df = pd.concat(dfs)
df.drop_duplicates(inplace=True)
df.drop(df.columns[0], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
%cd /home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA
df.to_hdf(f'TRN_navi_catalog.h5',key='df', format='table', complevel=9)

/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA


In [9]:
TRN_cat = pd.HDFStore('/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/TRN_navi_catalog.h5')

In [10]:
TRN_cat['df']

,Angle1,Angle2,Angle3,des1,des2,des3,lon1,lat1,r1,lon2,lat2,r2,lon3,lat3,r3
0,38.505996,17.784368,123.709636,2.061152,2.063950,7.891946,-65.477810,-44.911663,17.598053,-66.812988,-44.115738,6.351422,-67.501326,-44.443916,3.477863
1,123.709636,17.784368,38.505996,2.061152,7.891946,2.063950,-65.477810,-44.911663,17.598053,-67.501326,-44.443916,3.477863,-66.812988,-44.115738,6.351422
2,38.505996,123.709636,17.784368,2.063950,2.061152,7.891946,-66.812988,-44.115738,6.351422,-65.477810,-44.911663,17.598053,-67.501326,-44.443916,3.477863
3,17.784368,123.709636,38.505996,2.063950,7.891946,2.061152,-66.812988,-44.115738,6.351422,-67.501326,-44.443916,3.477863,-65.477810,-44.911663,17.598053
4,123.709636,38.505996,17.784368,7.891946,2.061152,2.063950,-67.501326,-44.443916,3.477863,-65.477810,-44.911663,17.598053,-66.812988,-44.115738,6.351422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7208536,111.466464,12.024531,56.509005,3.891215,8.438303,3.499912,118.351867,-56.859967,13.902800,120.419883,-58.803922,3.872122,119.801506,-58.949874,6.889018
7208537,56.509005,111.466464,12.024531,3.499912,3.891215,8.438303,119.801506,-58.949874,6.889018,118.351867,-56.859967,13.902800,120.419883,-58.803922,3.872122
7208538,12.024531,111.466464,56.509005,3.499912,8.438303,3.891215,119.801506,-58.949874,6.889018,120.419883,-58.803922,3.872122,118.351867,-56.859967,13.902800
7208539,111.466464,56.509005,12.024531,8.438303,3.891215,3.499912,120.419883,-58.803922,3.872122,118.351867,-56.859967,13.902800,119.801506,-58.949874,6.889018


In [11]:
TRN_cat['df']


,Angle1,Angle2,Angle3,des1,des2,des3,lon1,lat1,r1,lon2,lat2,r2,lon3,lat3,r3
0,38.505996,17.784368,123.709636,2.061152,2.063950,7.891946,-65.477810,-44.911663,17.598053,-66.812988,-44.115738,6.351422,-67.501326,-44.443916,3.477863
1,123.709636,17.784368,38.505996,2.061152,7.891946,2.063950,-65.477810,-44.911663,17.598053,-67.501326,-44.443916,3.477863,-66.812988,-44.115738,6.351422
2,38.505996,123.709636,17.784368,2.063950,2.061152,7.891946,-66.812988,-44.115738,6.351422,-65.477810,-44.911663,17.598053,-67.501326,-44.443916,3.477863
3,17.784368,123.709636,38.505996,2.063950,7.891946,2.061152,-66.812988,-44.115738,6.351422,-67.501326,-44.443916,3.477863,-65.477810,-44.911663,17.598053
4,123.709636,38.505996,17.784368,7.891946,2.061152,2.063950,-67.501326,-44.443916,3.477863,-65.477810,-44.911663,17.598053,-66.812988,-44.115738,6.351422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7208536,111.466464,12.024531,56.509005,3.891215,8.438303,3.499912,118.351867,-56.859967,13.902800,120.419883,-58.803922,3.872122,119.801506,-58.949874,6.889018
7208537,56.509005,111.466464,12.024531,3.499912,3.891215,8.438303,119.801506,-58.949874,6.889018,118.351867,-56.859967,13.902800,120.419883,-58.803922,3.872122
7208538,12.024531,111.466464,56.509005,3.499912,8.438303,3.891215,119.801506,-58.949874,6.889018,120.419883,-58.803922,3.872122,118.351867,-56.859967,13.902800
7208539,111.466464,56.509005,12.024531,8.438303,3.891215,3.499912,120.419883,-58.803922,3.872122,118.351867,-56.859967,13.902800,119.801506,-58.949874,6.889018


In [ ]:
# # def image_gen(PATH):
# path = 'DATA/ephemeris sat/20 step size/IMG_20_Lat_0_Long_-127.683.jpg'

# # Actual Position:
# num =20
# l = -127.683
# H = 50
# X_c, Y_c = l, 0
# # x_f, y_f, z_f = spherical2cartesian(0, crater.Lat, crater.Lon)

# # Find reference in catalogs:
# dteta = find_dteta(H) # d/R, where d^2 is the footprint
# sp = dteta/2 # span
# lat_bounds, lon_bounds = np.array([Y_c-sp,Y_c+sp]), np.array([X_c-sp,X_c+sp])
# print('*'*35 +'BORDERS'+'*'*35)
# print(f'LAT: {lat_bounds}              LON:{lon_bounds}')
# print('*'*35 +'*******'+'*'*35)
# # 3 crater catalogs: HEAD | LROC | ROBBINS
# CATALOGS = ['ROBBINS', 'HEAD', 'LROC']
# for DATA in CATALOGS:
#     if DATA =='HEAD':
#         filepath = '/DATA/HeadCraters.csv'
#     elif DATA == 'LROC':
#         filepath = '/DATA/LROCCraters.csv'
#     elif DATA == 'ROBBINS':
#         filepath= 'DATA/lunar_crater_database_robbins_2018.csv'

#     DB = pd.read_csv(filepath, sep=',')
#     df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME=DATA)
#     if df is None:
#         print('No Craters found!')
#     else:     
#         print(f'Found {df.shape[0]} craters!\n\n', df)
    
#     ## DRAWINGS: 
#     ## FIG.1
#     img = cv2.imread(path)
#     plt.ion()
#     plt.figure('Tile and Mask -'+DATA, dpi=200)
#     plt.subplot(1,2,1)
#     plt.imshow(img)
#     plt.subplot(1,2,2)
#     I = draw_craters(df, lon_bounds, lat_bounds)
#     plt.imshow(I)
#     plt.show()
#     plt.savefig(DATA+str(num)+'_fig-A.png')
    
#     ## FIG.2
#     image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, img, u=None)
#     plt.ion()
#     plt.figure('Image with craters catalogued-'+DATA, dpi=200)
#     plt.imshow(image_with_craters)
#     plt.show()
#     plt.savefig(DATA+str(num)+'_fig-B.png')

# plt.close('all')